In [ ]:
%load_ext autoreload
%autoreload 2

Install requirements from file or user input


In [ ]:
%%bash

#pip install pyspark polars pandas setuptools
pip install -r requirements.txt

In [ ]:
%%bash

pip list

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
.appName("Polars experiment") \
.getOrCreate()
spark

In [ ]:
path = "data/user-demographics.csv"

df_base = spark.read.option("delimiter", ",").option("header", True).csv(path)
#df_doubled = df_base.union(df_base)

In [ ]:
df_base.show()

In [ ]:
df_base.count()
#df_doubled.count()

In [ ]:
from datetime import datetime

In [ ]:
start_time = datetime.now()
end_time = datetime.now()
print(f"Time elapsed for processing: {end_time-start_time}")


Experiment spark df to pandas df

In [ ]:
# Workaround for pyspark 3.5.x with python 3.12 while not supported yet
import setuptools
from distutils.version import LooseVersion

In [ ]:
import pandas as pd

In [ ]:
%%time
start_time = datetime.now()

pd_base = df_base.toPandas()

end_time = datetime.now()
print(f"Time elapsed for processing spark to pandas df: {end_time-start_time}")

In [ ]:
pd_base

Generate Profile Report from Pandas Dataframe

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
%%time

start_time = datetime.now()

# Generate the profile report
report = ProfileReport(pd_base, title="Profile Report")

end_time = datetime.now()
print(f"Time elapsed for processing Report: {end_time-start_time}")

In [ ]:
report.to_notebook_iframe()

In [ ]:
import pyarrow as pa
import polars as pl

In [ ]:
%%time
arrow_temp = pa.Table.from_batches(df_base._collect_as_arrow())
#pl lazy dataframe
#df_polars = pl.scan_pyarrow_dataset(arrow_temp)
# pl dataframe
df_polars = pl.from_arrow(arrow_temp)


Experiment train kmeans model from polars lazy dataframe

In [ ]:
import numpy as np
from sklearn.cluster import KMeans


In [ ]:
df_polars.head()

In [ ]:
df_polars = df_polars.drop("Town/City","Country")

In [ ]:
# 1. Daten aus der Polars Tabelle extrahieren
# Wir wählen die Features aus, die für das Clustering verwendet werden sollen
#features = ["feature_1", "feature_2"]
data_for_clustering = df_polars.to_numpy()
df_polars


In [ ]:
# Alternativ, wenn Sie alle numerischen Spalten verwenden möchten:
# numeric_cols = pl_df.select(pl.col(pl.Float64)).columns
# data_for_clustering = pl_df.select(numeric_cols).to_numpy()

# 2. KMeans-Modell initialisieren und trainieren
n_clusters = 4  # Anzahl der gewünschten Cluster
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto') # Empfohlen: Setzen Sie random_state für Reproduzierbarkeit
kmeans.fit(data_for_clustering)



In [ ]:
# 3. Clusterzuordnungen erhalten
cluster_labels = kmeans.predict(data_for_clustering)



In [ ]:
# Die Cluster-Labels können Sie nun wieder zu Ihrer Polars Tabelle hinzufügen
df_polars = df_polars.with_columns(pl.Series("cluster", cluster_labels))

print(df_polars)
print(kmeans.cluster_centers_) # Anzeigen der Clusterzentroiden